In [2]:
#!pip install beautifulsoup4 requests
#!pip install nltk
#!pip install pandas transformers torch
#!pip install requests beautifulsoup4
#!pip install transformers
#!pip install torch


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL pour la catégorie materiaux-outils-equipements
URL = "https://www.expat-dakar.com/materiaux-outils-equipements"

# Headers pour simuler un navigateur
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Fonction pour récupérer les liens des annonces sur une page donnée
def get_announcement_links(url, page):
    response = requests.get(f"{url}?page={page}", headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extraire tous les liens des annonces
    links = []
    for item in soup.find_all("div", class_="listings-cards__list-item"):
        link_tag = item.find("a", href=True)
        if link_tag:
            href = link_tag['href']
            # Vérifiez que le lien est bien formaté
            if href.startswith("/"):  # Lien relatif
                links.append("https://www.expat-dakar.com" + href)
            elif href.startswith("http"):  # Lien absolu déjà formé
                links.append(href)

    return links

# Fonction pour récupérer les détails d'une annonce
def get_announcement_details(url):
    response = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(response.text, "html.parser")

    # Récupérer les informations spécifiques
    details = {}
    try:
        # Titre et prix
        details["Titre"] = soup.find("h1", class_="listing-item__header").text.strip() if soup.find("h1", class_="listing-item__header") else "N/A"
        details["Prix"] = soup.find("span", class_="listing-card__price__value").text.strip() if soup.find("span", class_="listing-card__price__value") else "N/A"

        # Adresse
        location = soup.find("span", class_="listing-item__address-location")
        region = soup.find("span", class_="listing-item__address-region")
        details["Adresse"] = f"{location.text.strip()} {region.text.strip()}" if location and region else "N/A"

        # Récupérer les détails listés dans "Détails"
        for dt, dd in zip(
            soup.find_all("dt", class_="listing-item__properties__title"),
            soup.find_all("dd", class_="listing-item__properties__description")
        ):
            details[dt.text.strip()] = dd.text.strip()

        # Description
        details["Description"] = soup.find("div", class_="listing-item__description").text.strip() if soup.find("div", class_="listing-item__description") else "N/A"
    except Exception as e:
        print(f"Erreur lors de l'extraction de {url}: {e}")

    return details

# Récupérer tous les liens et les détails des annonces sur plusieurs pages
print("Début du scraping des annonces de la catégorie materiaux outils equipements...")
all_links = []
data = []  # Pour stocker les détails des annonces

# Boucle pour parcourir les pages
MAX_PAGES = 500
for page in range(1, MAX_PAGES + 1):
    print(f"Scraping de la page {page}...")
    links = get_announcement_links(URL, page)
    all_links.extend(links)
    print(f"{len(links)} liens trouvés sur la page {page}.")

    # Récupérer les détails pour chaque annonce de cette page
    for link in links:
        print(f"Scraping des détails pour : {link}")
        details = get_announcement_details(link)
        data.append(details)
        time.sleep(1)  # Pause pour éviter de surcharger le serveur


# Sauvegarder les résultats dans un fichier CSV
df = pd.DataFrame(data)
df.to_csv("/content/drive/My Drive/Cas_etude_industriel/annonces_materiaux_outils_equipements.csv", index=False)
print("Les données ont été sauvegardées dans 'annonces_materiaux_outils_equipements.csv'.")



Début du scraping des annonces de la catégorie materiaux outils equipements...
Scraping de la page 1...
10 liens trouvés sur la page 1.
Scraping des détails pour : https://www.expat-dakar.com/annonce/imou-bulb-cam-3mp-2k-camera-wifi-6027582
Scraping des détails pour : https://www.expat-dakar.com/annonce/faux-plafond-en-bois-6242400
Scraping des détails pour : https://www.expat-dakar.com/annonce/refrigerateur-6242217
Scraping des détails pour : https://www.expat-dakar.com/annonce/machine-multifonction-petit-dej-6242264
Scraping des détails pour : https://www.expat-dakar.com/annonce/deambulateur-bequilles-de-qualite-6191015
Scraping des détails pour : https://www.expat-dakar.com/annonce/chaise-fauteuil-daisance-douche-et-toilettes-6242258
Scraping des détails pour : https://www.expat-dakar.com/annonce/puff-cbd-650-bouffe-6242199
Scraping des détails pour : https://www.expat-dakar.com/annonce/puff-reutilisable-x-bar-v2-650-bouffe-6228551
Scraping des détails pour : https://www.expat-dakar

In [7]:
print(df)

                                    Titre           Prix              Adresse  \
0       Imou Bulb Cam 3MP 2k  caméra wifi   37 500 F Cfa        Plateau Dakar   
1                    Faux plafond en bois   13 000 F Cfa      Liberte 6 Dakar   
2                           Réfrigérateur  800 000 F Cfa       Yeumbeul Dakar   
3         Machine multifonction petit dej   30 000 F Cfa    Djily mbaye Dakar   
4     Déambulateur & béquilles de qualité   53 000 F Cfa  Hann maristes Dakar   
...                                   ...            ...                  ...   
1782               Thermomètre infrarouge        1 F Cfa        Plateau Dakar   
1783             Câble 24 Brin  multimode    3 000 F Cfa   Gueule-Tapée Dakar   
1784                     Jeux de lumières        1 F Cfa        Plateau Dakar   
1785             Caméra endescope en tube   15 000 F Cfa        Plateau Dakar   
1786             LOT DE BARQUETTE 500 GRS   12 500 F Cfa  Hann maristes Dakar   

                           